In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate 
from langchain.chains.llm import LLMChain
from cot import Collection
import json

In [2]:
"""Chain to retrieve a chain-of-thought"""
llm = OpenAI(temperature=.0)
template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

{cot_trigger}
"""
prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger"], template=template)
cot_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="cot")

In [3]:
"""This chain retrieves answer extraction"""
extraction_template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

Cot: {cot_trigger}{cot}
{answer_extraction}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger","cot","answer_extraction"], template=extraction_template)
answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="answer")


In [6]:
"""Combine chains"""

from langchain.chains import SequentialChain
overall_chain = SequentialChain(chains=[cot_chain, answer_chain],
                                input_variables=["instruction","question","answer_choices","cot_trigger","answer_extraction"],
                                output_variables=["cot", "answer"],
                                verbose=True)

In [4]:
input_dict = {
    "instruction": "Be faithful and a little hopeful",
    "cot_trigger": "Answer: Let's think step by step.",
    "answer_extraction_key": ['kojima-A-D'], 
}


worldtree = Collection(["worldtree"], verbose=False)
worldtree_1 = worldtree.select(split="train", number_samples=1, random_samples=True, seed=0)

Loading worldtree...


In [7]:
"""generate_extract_flexibly in dataloader.py calls self_generate_extract in new_generation.py
The problem is in line 36 of new_generation.py the SequentialChain which reuses parts 
of the templates for the chains is part of the input dict (kwargs) - data.map
does not seem to be able to handle a nested dict.

The problem is that fn_kwargs takes inputs (fn_kwargs={"tokenizer":tokenizer}),
 we don't want that since we want to build the chain before calling data.map and not after
"""
test = worldtree_1.generate_extract_flexibly(chain=overall_chain,input_dict=input_dict)

Generating worldtree...


TypeError: _self_generate_extract() got multiple values for argument 'instruction'